In [1]:
import ast
import json
import os
import shutil
import albumentations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom as dicom
import tensorflow as tf
import torch
from sklearn.model_selection import StratifiedKFold
from PIL import Image
from tensorflow import keras
from tqdm import tqdm
from src.utils import Aug, Generator, GetModel
import tensorflow_addons as tfa

In [2]:
df = pd.read_csv("/app/_data/train.csv")

In [3]:
with open("/app/_data/base_config.json", "r") as f:
    base_config = json.load(f)
base_config

{'EFFB7': {'IMG_SIZE': 600,
  'BATCH_SIZE': 16,
  'SEED': 42,
  'WEIGHTS': '/app/_data/efficientnet-b7_noisy-student_notop.h5'},
 'CLASS': {'negative': 0, 'typical': 1, 'indeterminate': 2, 'atypical': 3}}

In [4]:
base_config['EFFB7']["BATCH_SIZE"]=4
base_config

{'EFFB7': {'IMG_SIZE': 600,
  'BATCH_SIZE': 4,
  'SEED': 42,
  'WEIGHTS': '/app/_data/efficientnet-b7_noisy-student_notop.h5'},
 'CLASS': {'negative': 0, 'typical': 1, 'indeterminate': 2, 'atypical': 3}}

In [5]:
skf = StratifiedKFold(n_splits=5, random_state=base_config['EFFB7']["SEED"], shuffle=True)
train_ids = []
val_ids = []
for train_index, valid_index in skf.split(df, df["class"]):
    train_ids.append(train_index)
    val_ids.append(valid_index)

In [6]:
# shutil.rmtree('/app//_data/train_jpg_600/')

In [7]:
gen_train = Generator(
    df=df.loc[train_ids[0]],
    batch_size=base_config['EFFB7']["BATCH_SIZE"],
    seed=base_config['EFFB7']["SEED"],
    img_size=base_config['EFFB7']["IMG_SIZE"],
    prepared_img_path="/app/_data/train_jpg_600/",
    shuffle =True,
    augment = True
)
gen_valid = Generator(
    df=df.loc[val_ids[0]],
    batch_size=base_config['EFFB7']["BATCH_SIZE"],
    seed=base_config['EFFB7']["SEED"],
    img_size=base_config['EFFB7']["IMG_SIZE"],
    prepared_img_path="/app/_data/train_jpg_600/",
    shuffle =False,
    augment = False
)

In [8]:
train_sample = next(iter(gen_train))

In [9]:
# plt.figure(figsize = (15,8))
# for i in range(8):
#     plt.subplot(2,4,i+1)
#     plt.title(train_sample[1][i])
#     plt.imshow(train_sample[0]['img'][i])

In [10]:
model = GetModel("EFFB7").get_model()

In [11]:
callbacks = GetModel("EFFB7").make_callback(
    model_path="/app/_data/models/EffB7_init/",
    model_name = 'EffB7_0.h5',
    tensorboard_path="/app/.tensorboard/EffB7_init_0",
    patience_ES=12,
    patience_RLR=5,
    factor_LR=0.9,
    metric_for_monitor="val_loss",
    metric_mode="min",
)

Warning! Model path already exists.


In [ ]:
history = model.fit(
    gen_train,
    validation_data=gen_valid,
    epochs=100,
    steps_per_epoch=len(train_ids[0]) // base_config['EFFB7']["BATCH_SIZE"]-1,
    validation_steps=len(val_ids[0]) // base_config['EFFB7']["BATCH_SIZE"]-1,
    verbose=1,
    workers=20,
    max_queue_size=1000,
    callbacks=callbacks,
)

Epoch 1/100
